In [5]:
import requests

API_URL = "http://taketwo-webapi-route-embrace-dev-js.embrace-dev-ocp43-vpc-7ec5d722a0ab3f463fdc90eeb94dbc70-0000.us-east.containers.appdomain.cloud/"

In [1]:
pwd

'/Users/johannasaladas/Documents/taketwo/taketwo-datascience'

In [33]:
response = requests.get(API_URL + "categories")
if response.status_code == 200:
    categories = [category["name"] for category in response.json()]
    categories += ["racial slur"] #add additional not in the API manually

In [40]:
categories

['appropriation',
 'stereotyping',
 'deflection',
 'gaslighting',
 'othering',
 'racial slur']

In [13]:
 from sklearn.datasets import fetch_20newsgroups

In [17]:
twenty_train = fetch_20newsgroups(subset='train',
...     categories=categories, shuffle=True, random_state=42)

In [26]:
list(twenty_train.keys())

['data', 'filenames', 'target_names', 'target', 'DESCR']

In [42]:
response = requests.get(API_URL + "mark")
if response.status_code == 200:
    raw_data = response.json()

In [57]:

from collections import namedtuple
data_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

for row in raw_data:
    if row['category'] in categories:
        data_set.data.append(row["flagged_string"])
        data_set.urls.append(row["url"])
        data_set.target.append(categories.index(row['category']))

In [58]:
data_set

data_set(data=['zigabo', 'niggah', '#WhiteLivesMatter', 'Golliwogg', 'niggah', 'pre-pandemic', 'black up', 'coronavirus', 'jigg', 'niggress', 'carrying on a “unique mining tradition”', 'golliwog', 'jiggabo', 'Mainland-listed technology', 'ahead', 'colored', 'fuzzy-wuzzy', 'golliwogg', 'niggette', 'slaves to royalty', 'send them all back', 'Sed finibus elit vulputate erat vehicula, eget fermentum lectus feugiat. ', 'refusing to stop blacking up their faces', 'boogie', 'coon', 'pickaninny', "all that ‘jigaboo' music", 'The New York', 'curtains', 'bouncing', 'jig', 'nigger', 'nigga', 'nigguh', 'niggar', 'coloured', 'straight outta Compton', 'Why don’t they go back and help fix the totally broken and crime infested places from which they came.', 'prison', 'focus', 'packed plane', 'fun than economy', 'Felis donec et odio pellentesque', 'a elit sit amet', 'palace required revisions', ' up to 15 years', 'nigg', 'Markle grew up in an area that was home to the Bloods gang', 'Stop crying about r

In [64]:
print(data_set.target_names[data_set.target[0]])

racial slur


In [65]:
data_set.target

[5,
 5,
 3,
 5,
 5,
 1,
 0,
 2,
 5,
 5,
 0,
 5,
 5,
 3,
 1,
 5,
 5,
 5,
 5,
 1,
 4,
 3,
 1,
 5,
 5,
 5,
 5,
 2,
 4,
 0,
 5,
 5,
 5,
 5,
 5,
 5,
 1,
 4,
 1,
 0,
 2,
 3,
 2,
 2,
 3,
 0,
 5,
 1,
 3,
 1,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 2,
 3,
 1,
 5,
 5]

In [66]:
len(data_set.target)

64

In [67]:
for t in data_set.target[:10]:
...     print(data_set.target_names[t])

racial slur
racial slur
gaslighting
racial slur
racial slur
stereotyping
appropriation
deflection
racial slur
racial slur


In [48]:
from sklearn.feature_extraction.text import CountVectorizer

In [49]:
count_vect = CountVectorizer()

In [59]:
X_train_counts = count_vect.fit_transform(data_set.data)

In [60]:
 X_train_counts.shape

(64, 141)

In [19]:
 X_train_counts

<2257x35788 sparse matrix of type '<class 'numpy.int64'>'
	with 365886 stored elements in Compressed Sparse Row format>

In [20]:
count_vect.vocabulary_.get(u'algorithm')

4690

In [21]:
from sklearn.feature_extraction.text import TfidfTransformer
>>> tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
>>> X_train_tf = tf_transformer.transform(X_train_counts)
>>> X_train_tf.shape

(2257, 35788)

In [23]:
>>> tfidf_transformer = TfidfTransformer()
>>> X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
>>> X_train_tfidf.shape
(2257, 35788)

(2257, 35788)

In [24]:
from sklearn.naive_bayes import MultinomialNB
>>> clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [25]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
>>> X_new_counts = count_vect.transform(docs_new)
>>> X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [26]:
X_new_tfidf

<2x35788 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [27]:
predicted = clf.predict(X_new_tfidf)

In [28]:
for doc, category in zip(docs_new, predicted):
...     print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


In [29]:
from sklearn.pipeline import Pipeline
>>> text_clf = Pipeline([
...     ('vect', CountVectorizer()),
...     ('tfidf', TfidfTransformer()),
...     ('clf', MultinomialNB()),
... ])

In [31]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [32]:
import numpy as np
>>> twenty_test = fetch_20newsgroups(subset='test',
...     categories=categories, shuffle=True, random_state=42)
>>> docs_test = twenty_test.data
>>> predicted = text_clf.predict(docs_test)
>>> np.mean(predicted == twenty_test.target)

0.8348868175765646

In [33]:
from sklearn.linear_model import SGDClassifier
>>> text_clf = Pipeline([
...     ('vect', CountVectorizer()),
...     ('tfidf', TfidfTransformer()),
...     ('clf', SGDClassifier(loss='hinge', penalty='l2',
...                           alpha=1e-3, random_state=42,
...                           max_iter=5, tol=None)),
... ])

In [34]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                      

In [35]:
predicted = text_clf.predict(docs_test)

In [36]:
 np.mean(predicted == twenty_test.target)

0.9101198402130493

In [37]:
predicted

array([2, 2, 2, ..., 2, 2, 1])

In [38]:
predicted[:10]

array([2, 2, 2, 0, 3, 0, 1, 3, 1, 2])

In [39]:
twenty_test.target[:10]

array([2, 2, 2, 0, 3, 0, 1, 3, 2, 2])

In [40]:
predicted[:50]

array([2, 2, 2, 0, 3, 0, 1, 3, 1, 2, 1, 3, 0, 3, 1, 2, 1, 3, 0, 3, 2, 3,
       1, 0, 2, 1, 1, 3, 2, 0, 1, 3, 0, 2, 2, 0, 0, 0, 2, 1, 2, 0, 2, 3,
       2, 2, 3, 1, 3, 1])

In [41]:
twenty_test.target[:50]

array([2, 2, 2, 0, 3, 0, 1, 3, 2, 2, 1, 3, 2, 3, 1, 0, 1, 3, 0, 0, 2, 3,
       1, 0, 2, 1, 1, 3, 2, 0, 1, 3, 0, 2, 2, 0, 2, 0, 2, 1, 2, 0, 2, 3,
       2, 2, 3, 1, 3, 1])

In [42]:
from sklearn import metrics
>>> print(metrics.classification_report(twenty_test.target, predicted,
...     target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.80      0.87       319
         comp.graphics       0.87      0.98      0.92       389
               sci.med       0.94      0.89      0.91       396
soc.religion.christian       0.90      0.95      0.93       398

              accuracy                           0.91      1502
             macro avg       0.91      0.91      0.91      1502
          weighted avg       0.91      0.91      0.91      1502



In [43]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[256,  11,  16,  36],
       [  4, 380,   3,   2],
       [  5,  35, 353,   3],
       [  5,  11,   4, 378]])

In [44]:
from sklearn.model_selection import GridSearchCV
>>> parameters = {
...     'vect__ngram_range': [(1, 1), (1, 2)],
...     'tfidf__use_idf': (True, False),
...     'clf__alpha': (1e-2, 1e-3),
... }

In [45]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

In [47]:
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

/Users/naokieabe/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [48]:
twenty_train.target_names[gs_clf.predict(['God is love'])[0]]

'soc.religion.christian'

In [49]:
for param_name in sorted(parameters.keys()):
...     print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 1)
